# MIMIC-III query API tutorial

This notebook shows examples of how to use the cyclops.query API on [MIMIC-III v1.4](https://physionet.org/content/mimiciii/1.4/).

* First, setup the MIMIC-III database according to the instructions in [mimic-code](https://github.com/MIT-LCP/mimic-code/tree/main/mimic-iii/buildmimic/postgres).
* The database is assumed to be hosted using postgres. Update the config parameters such as username and password, passed to `MIMICIIIQuerier` accordingly.

## Imports and instantiate `MIMICIIIQuerier`

In [1]:
import cyclops.query.ops as qo
from cyclops.query import MIMICIIIQuerier

mimic = MIMICIIIQuerier(
    dbms="postgresql",
    port=5432,
    host="localhost",
    database="mimiciii",
    user="postgres",
    password="pwd",
)
# List all tables.
mimic.list_tables()

2023-03-07 12:53:36,501 INFO cyclops.query.orm - Database setup, ready to run queries!


['patients',
 'admissions',
 'diagnoses_icd',
 'labevents',
 'chartevents',
 'transfers',
 'prescriptions',
 'microbiologyevents',
 'procedureevents_mv',
 'dim_labitems',
 'dim_items',
 'dim_icd_diagnoses']

## Example 1. Get all male patients with a mortality outcome.

In [2]:
ops = qo.Sequential(
    [
        qo.ConditionEquals("expire_flag", 1),
        qo.ConditionEquals("gender", "M"),
    ]
)
patients = mimic.patients(ops=ops).run()
print(f"{len(patients)} rows extracted!")

2023-03-07 12:53:37,064 INFO cyclops.query.orm - Query returned successfully!
2023-03-07 12:53:37,065 INFO cyclops.utils.profile - Finished executing function run_query in 0.535097 s


8524 rows extracted!


## Example 2. Get all female patient encounters with diagnoses (`gastroenteritis` in ICD-9 long title).

In [3]:
ops = qo.Sequential(
    [
        qo.ConditionEquals("gender", "F"),
    ]
)
diagnoses_ops = qo.Sequential(
    [
        qo.ConditionSubstring("long_title", "gastroenteritis"),
    ]
)
patients = mimic.patients(ops=ops)
admissions = mimic.admissions(
    join=qo.JoinArgs(join_table=patients.query, on="subject_id"),
)
diagnoses = mimic.diagnoses_icd(
    join=qo.JoinArgs(
        join_table=admissions.query,
        on=["subject_id", "hadm_id"],
    ),
    ops=diagnoses_ops,
).run()
print(f"{len(diagnoses)} rows extracted!")

2023-03-07 12:54:04,751 INFO cyclops.query.orm - Query returned successfully!
2023-03-07 12:54:04,752 INFO cyclops.utils.profile - Finished executing function run_query in 26.071373 s


162 rows extracted!


## Example 3. Get potassium lab tests for female patient encounters.

In [4]:
ops = qo.Sequential(
    [
        qo.ConditionEquals("gender", "F"),
    ]
)
patients = mimic.patients(ops=ops)
lab_ops = qo.Sequential(
    [
        qo.ConditionEquals("label", "potassium"),
    ]
)
labs = mimic.labevents(ops=lab_ops).run()
print(f"{len(labs)} rows extracted!")

2023-03-07 12:54:31,969 INFO cyclops.query.orm - Query returned successfully!
2023-03-07 12:54:31,970 INFO cyclops.utils.profile - Finished executing function run_query in 27.185277 s


845739 rows extracted!


## Example 4. Get AaDO2 carevue chart events for male patients that have a `valuenum` of less than 20.

In [5]:
ops = qo.Sequential(
    [
        qo.ConditionEquals("gender", "M"),
    ]
)
chartevents_ops = qo.Sequential(
    [
        qo.ConditionEquals("dbsource", "carevue"),
        qo.ConditionEquals("label", "AaDO2"),
        qo.ConditionLessThan("valuenum", 20),
    ]
)
patients = mimic.patients(ops=ops)
chart_events = mimic.chartevents(ops=chartevents_ops).run()
print(f"{len(chart_events)} rows extracted!")

2023-03-07 12:55:54,659 INFO cyclops.query.orm - Query returned successfully!
2023-03-07 12:55:54,660 INFO cyclops.utils.profile - Finished executing function run_query in 82.670165 s


13 rows extracted!
